In [5]:
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF/'
save_folder_path = '/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr-reading/visualizations/'
timestep_range = range(6)
dimension = 'z'

In [ ]:
!pip install "xarray[complete]"
!pip install "dask[complete]"
!pip install Pillow

In [ ]:
# needed for sciserver jobs
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_reading

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from utils import visualization_tools
import dask

In [ ]:
tasks = []

for timestep_nr in timestep_range:
    ncar_data = xr.open_dataset(raw_ncar_folder_path + "/jhd.00" + str(timestep_nr) + ".nc")
    for var in ncar_data.data_vars:
        # Wrap the function call with dask.delayed
        task = dask.delayed(visualization_tools.animate_cube)(ncar_data[var], dimension=dimension, save_filename=save_folder_path + "timestep_" + str(timestep_nr) + "/" + var)
        tasks.append(task)

# Use dask.compute to execute all tasks in parallel
results = dask.compute(*tasks)

In [ ]:
# def process_var(var):
#     visualization_tools.animate_cube(ncar_data[var], dimension=dimension, save_filename="/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_reading/visualizations/timestep_" + str(timestep_nr) + "/" + var)

In [3]:
# for timestep_nr in timestep_range:
#     ncar_data = xr.open_dataset(raw_ncar_folder_path + "jhd." + str(timestep_nr).zfill(3) + ".nc")
#     with concurrent.futures.ProcessPoolExecutor(6) as executor:
#         executor.map(process_var, ncar_data.data_vars)

In [ ]:
# for var in ncar_data.data_vars:
#     visualization_tools.animate_cube(ncar_data[var], dimension=dimension, save_filename="visualizations/timestep_0/" + var)